In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error

In [13]:
 # load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('ML-CUP23-TR.csv', delimiter=',')
X = dataset[:,1:11]
y = dataset[:,11:14]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# Define a simple regression neural network
class RegressorNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressorNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Set hyperparameters
input_size = 10
output_size = 3

hidden_sizes = [10,20,30]
learning_rates = [0.01,0.001]
epochs = 300

best_loss = 1e5 #initialize to high value
best_hyperparams = []

for hidden_size, learning_rate in product(hidden_sizes,learning_rates):
    # Define the model, loss function, and optimizer
    model = RegressorNN(input_size, hidden_size, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Define K-fold cross-validation
    k_folds = 5
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Lists to store training and validation loss for each epoch
    train_losses = []
    val_losses = []

    # Perform K-fold cross-validation
    for fold, (train_indices, val_indices) in enumerate(kf.split(X)):
        #print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data into training and validation sets
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]

        # Training loop
        for epoch in range(epochs):
            # Forward pass
            outputs = model(X_train)
            loss = criterion(outputs, y_train)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #if (epoch + 1) % 100 == 0:
            #    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

        # Save training loss for plotting
        train_losses.append(loss.item())

        # Validation
        with torch.no_grad():
            model.eval()
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
            #mse = mean_squared_error(y_val.numpy(), val_outputs.numpy())
            #print(f"Validation MSE: {mse:.4f}")
            #print(f"Validation Loss: {val_loss.item():.4f}")

            # Save validation loss for plotting
            val_losses.append(val_loss.item())

        model.train()  # Set the model back to training mode

    print(f'\nHyperparams: hidden_size={hidden_size}; lr={learning_rate}')
    print(f'train_loss = {np.mean(train_losses):.4} +- {np.std(train_losses):.4} | '
          f'val_loss = {np.mean(val_losses):.4} +- {np.std(val_losses):.4}')

    if np.mean(val_losses) < best_loss:
        best_loss = np.mean(val_losses)
        best_hyperparams = [hidden_size, learning_rate]

print(best_hyperparams)


Hyperparams: hidden_size=10; lr=0.01
train_loss = 12.08 +- 11.06/ val_loss = 9.271 +- 6.538

Hyperparams: hidden_size=10; lr=0.001
train_loss = 14.86 +- 9.701/ val_loss = 14.61 +- 8.427

Hyperparams: hidden_size=20; lr=0.01
train_loss = 7.123 +- 6.339/ val_loss = 6.423 +- 4.958

Hyperparams: hidden_size=20; lr=0.001
train_loss = 8.403 +- 5.488/ val_loss = 8.487 +- 5.213

Hyperparams: hidden_size=30; lr=0.01
train_loss = 5.057 +- 4.297/ val_loss = 5.065 +- 4.504
